In [3]:
# Código para analizar patrones dentro de denuncias clasificadas como "Otro"

import pandas as pd
import re
from collections import Counter
import altair as alt

# Subir archivo desde el equipo
from google.colab import files
uploaded = files.upload()

# Cargar el archivo Excel
df = pd.read_excel(next(iter(uploaded)))

# Renombrar columnas claves
df = df.rename(columns={
    'CANAL / CABLE ESTANDARIZADO': 'Canal',
    'Nº Denuncias': 'Denuncias',
    'Contenidos Denunciados': 'Contenido',
    'AÑO ': 'Año',
    'Programa': 'Programa'
})

# Limpiar textos
df['Contenido'] = df['Contenido'].fillna('').str.lower()

# Clasificación temática
def clasificar_tema(texto):
    if any(p in texto for p in ['violencia', 'agresión', 'agresivo', 'golpe', 'maltrato', 'amenaza', 'ataque', 'abuso']):
        return 'Violencia, Agresión y Conductas Inapropiadas'
    elif any(p in texto for p in ['dignidad', 'intimidad', 'vida privada', 'exposición indebida', 'humillación']):
        return 'Dignidad / Intimidad / Vida Privada'
    elif any(p in texto for p in ['sexual', 'desnudo', 'vulgar', 'lenguaje soez', 'contenido sexual']):
        return 'Contenido Sexual / Lenguaje Vulgar'
    elif any(p in texto for p in ['manipulación', 'sesgo', 'desinformación', 'falta de pluralismo', 'falsedad']):
        return 'Desinformación / Sesgo / Falta de Pluralismo / Manipulación'
    elif any(p in texto for p in ['niño', 'niñez', 'menor', 'horario de protección', 'infantil']):
        return 'Niñez / Horario de Protección'
    else:
        return 'Otro'

# Clasificar
df['Tema Clasificado'] = df['Contenido'].apply(clasificar_tema)

# Filtrar solo denuncias clasificadas como "Otro"
otros_df = df[df['Tema Clasificado'] == 'Otro']

# Definir patrones para encontrar temas recurrentes
posibles_temas = {
    'Discriminación': ['discriminación', 'discrimina', 'racismo', 'xenofobia', 'estigmatiza'],
    'Religión': ['evangélico', 'religión', 'católico', 'cristiano', 'iglesia'],
    'Política': ['político', 'política', 'presidente', 'congreso', 'campaña'],
    'Ética periodística': ['ética', 'sensacionalista', 'manipulado', 'exagerado'],
    'Estereotipos de género': ['machismo', 'misógino', 'estereotipo', 'sexista'],
    'Tratamiento de la muerte': ['suicidio', 'muerte', 'fallecimiento', 'morbo']
}

# Clasificar según estos nuevos patrones
def clasificar_otro(texto):
    for tema, palabras in posibles_temas.items():
        if any(p in texto for p in palabras):
            return tema
    return 'No clasificado'

otros_df['Tema Otro Detectado'] = otros_df['Contenido'].apply(clasificar_otro)

# Contar ocurrencias
tema_counts = otros_df['Tema Otro Detectado'].value_counts().reset_index()
tema_counts.columns = ['Tema Detectado', 'Cantidad']

print("\nPatrones detectados dentro de las denuncias clasificadas como 'Otro':\n")
print(tema_counts)

# Mostrar ejemplos por tema
for tema in tema_counts['Tema Detectado']:
    print(f"\nEjemplos de denuncias clasificadas como '{tema}':")
    ejemplos = otros_df[otros_df['Tema Otro Detectado'] == tema]['Contenido'].head(3)
    for i, ejemplo in enumerate(ejemplos, 1):
        print(f"  {i}. {ejemplo[:200]}...")

# Visualización con Altair
chart = alt.Chart(tema_counts).mark_bar().encode(
    x=alt.X('Cantidad:Q', title='Cantidad de denuncias'),
    y=alt.Y('Tema Detectado:N', sort='-x', title='Tema Detectado'),
    color=alt.Color('Tema Detectado:N', legend=None)
).properties(
    title='Patrones emergentes dentro de las denuncias clasificadas como "Otro"',
    width=700,
    height=400
)

chart

Saving Denuncias_Clasificadas_Personajes (1).xlsx to Denuncias_Clasificadas_Personajes (1) (2).xlsx

Patrones detectados dentro de las denuncias clasificadas como 'Otro':

             Tema Detectado  Cantidad
0            No clasificado       189
1            Discriminación        32
2                  Política        19
3        Ética periodística        12
4  Tratamiento de la muerte        11
5                  Religión         7

Ejemplos de denuncias clasificadas como 'No clasificado':
  1. entrevista a parasicólogo que pronostica nuevo terremoto promueve el pánico en la comunidad....
  2. rutina humorística que ofende a la comunidad evangélica...
  3. predicciones de nuevos terremotos e informaciones sobre procedencia de alimentos provocan temor en la comunidad....

Ejemplos de denuncias clasificadas como 'Discriminación':
  1. programa criminaliza y estigmatiza personas que cultivan el hip hop...
  2. rutina humorística ofende y discrimina a comunidad evangélica....
  3. estigm

/tmp/ipython-input-3-2173001337.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  otros_df['Tema Otro Detectado'] = otros_df['Contenido'].apply(clasificar_otro)


alt.Chart(...)